In [1]:
import pandas as pd
import numpy as np

In [2]:
data_frame=pd.read_csv("data\medical_conversations.csv")

<>:1: SyntaxWarning: invalid escape sequence '\m'
<>:1: SyntaxWarning: invalid escape sequence '\m'
C:\Users\GUNATEJA\AppData\Local\Temp\ipykernel_7400\880099130.py:1: SyntaxWarning: invalid escape sequence '\m'
  data_frame=pd.read_csv("data\medical_conversations.csv")


In [3]:
data_frame.head()

,conversations,disease
0,User: I’ve been sneezing a lot today and my no...,allergy
1,User: I’ve developed a rash after eating some ...,allergy
2,"User: My eyes are swollen and itchy, and I can...",allergy
3,User: I’ve been getting headaches and a stuffy...,allergy
4,"User: Every time I eat nuts, my mouth itches. ...",allergy


In [4]:
from bs4 import BeautifulSoup
import re
import pandas as pd

df = data_frame

def preprocess_text(text):
    if pd.notnull(text) and isinstance(text, str):
        text = BeautifulSoup(text, "html.parser").get_text()
        

        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
        
 
        text = text.lower()
    return text


columns_to_clean = ['conversations']


for column in columns_to_clean:
    if column in df.columns:  
        df[column] = df[column].apply(preprocess_text)


In [5]:
data_frame['conversations'][0]

'user ive been sneezing a lot today and my nose feels congested  bot that sounds like it could be an allergy do you know what might be triggering it  user im not sure maybe pollen  bot pollen is a common allergen have you had any other symptoms like itchy eyes or a sore throat  user yes my eyes are itchy too  bot sounds like classic hay fever overthecounter antihistamines could help relieve the symptoms '

In [25]:
data_frame.iloc[599]

conversations    user ive trouble sleeping neck pain tip sleepi...
disease                                       cervical spondylosis
Name: 599, dtype: object

In [6]:
import nltk
from  nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords

In [7]:
lemmatizer=WordNetLemmatizer()
for i in range(len(data_frame)):
    text = data_frame['conversations'][i]
    
    if isinstance(text, str):
        text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
        text = text.lower()
        words = text.split()
        words = [lemmatizer.lemmatize(word) for word in words if word not in stopwords.words('english')]
        data_frame.at[i, 'conversations'] = ' '.join(words)

In [9]:
from nltk.tokenize import sent_tokenize
for i in range(len(data_frame)):
    data_frame['conversations'][i]=sent_tokenize(data_frame['conversations'][i])

C:\Users\GUNATEJA\AppData\Local\Temp\ipykernel_7400\1880862408.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data_frame['conversations'][i]=sent_tokenize(data_frame['conversations'][i])
C:\Users\GUNATEJA\AppData\Local\Temp\ipykernel_74

In [10]:
from  sklearn.feature_extraction.text import TfidfVectorizer
tfid = TfidfVectorizer(stop_words='english', ngram_range=(1, 3))
data_frame['conversations'] = data_frame['conversations'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
tfid.fit(data_frame['conversations'])


TfidfVectorizer(ngram_range=(1, 3), stop_words='english')

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
tdf=TfidfVectorizer(stop_words='english',ngram_range=(1,3))
label=LabelEncoder()
x=tdf.fit_transform(data_frame['conversations'])
y=label.fit_transform(data_frame['disease'])
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [14]:
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)
print("Sample of y_train:", y_train[:10])
print("Sample of y_test:", y_test[:10])

y_train shape: (768,)
y_test shape: (192,)
Sample of y_train: [22 22  3  3 21 23 23 20 10  8]
Sample of y_test: [ 6 17  1  9  3 12  8  4 11 20]


In [15]:
y_train = y_train.ravel()
y_test = y_test.ravel()

In [16]:
from sklearn.model_selection import train_test_split
X = data_frame['conversations'].fillna("").tolist()
y = data_frame['disease'] 
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
X = data_frame['conversations'].fillna("").tolist()
y = data_frame['disease'] 
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
tfidf_vectorizer = TfidfVectorizer()
x_train_tfidf = tfidf_vectorizer.fit_transform(x_train)  
x_test_tfidf = tfidf_vectorizer.transform(x_test) 

In [18]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score
model = BernoulliNB()
model.fit(x_train_tfidf, y_train)
train_accuracy = accuracy_score(y_train, model.predict(x_train_tfidf))
print(f"mOdEl accuracy: {train_accuracy * 100}%") 

mOdEl accuracy: 99.47916666666666%


In [19]:
from sklearn.naive_bayes import MultinomialNB
model2=MultinomialNB()
model2.fit(x_train_tfidf, y_train)
train_accuracy = accuracy_score(y_train, model2.predict(x_train_tfidf))
print(f"accuracy_score is :{train_accuracy*100}%")

accuracy_score is :99.21875%


In [20]:
import pickle
pickle.dump(model,open('minidoc.pkl','wb'))
model=pickle.load(open('minidoc.pkl','rb'))

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
tfidf_vectorizer8 = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer8.fit_transform(x_train)
with open("tfidf_vectorizer8.pkl", "wb") as f:
    pickle.dump(tfidf_vectorizer8, f)